# Creating problems 
In this notebook, we show two ways for defining optimization problems in MetaJul. We focus here in continuous problems, but the same approaches can be applied to other kind of problems.

A continuous problem is characterized by

* A vector of ``Bounds`` objects, specifying the lower and upper bounds of the decision variables
* A vector containing the objective values
* A vector contining the overall constraint violation values (not used for now)
* The problem name

so the point is how to create a problem including these elements. Both approaches make use of ``createSolution()`` function that creates a solution with its variables initialized randomly from the problem bounds:

```julia
function createSolution(problem::AbstractContinuousProblem{T})::ContinuousSolution{T} where {T<:Number}
  x = [problem.bounds[i].lowerBound + rand() * (problem.bounds[i].upperBound - problem.bounds[i].lowerBound) for i in 1:length(problem.bounds)]

  return ContinuousSolution{T}(x, zeros(numberOfObjectives(problem)), zeros(numberOfConstraints(problem)), Dict(), problem.bounds)
end
```

## Method 1: Using the struct ``ContinuousProblem``

The first approach is to create them in a dynamic way by creating an instance of the ``ContinuousProblem`` struct defined in ``src/continuousProblem.jl``:

```julia
abstract type AbstractContinuousProblem{T<:Number} <: Problem{T} end

mutable struct ContinuousProblem{T} <: AbstractContinuousProblem{T}
  bounds::Vector{Bounds{T}}
  objectives::Vector{Function}
  constraints::Vector{Function}
  name::String
end
```

Let us suppose that we intend to implement problem Schaffer:

$$
f_1(x) = x 
$$
$$
f_2(x) = (x-2)^2
$$
$$
x \in [-10^3, 10^3]
$$

We can create the problem in this way:
```julia
  schaffer = ContinuousProblem{Real}("Schaffer")

  f = x -> x[1] * x[1]
  g = x -> (x[1] - 2.0) * (x[1] - 2.0)

  addObjective(schaffer, f)
  addObjective(schaffer, g)
  addVariable(schaffer, Bounds{Real}(-1000.0, 1000.0))
```

In [10]:
include("../src/continuousProblem.jl")

schaffer = ContinuousProblem{Real}("Schaffer")

f = x -> x[1] * x[1]
g = x -> (x[1] - 2.0) * (x[1] - 2.0)

addObjective(schaffer, f)
addObjective(schaffer, g)
addVariable(schaffer, Bounds{Real}(-1000.0, 1000.0))

println(schaffer)

ContinuousProblem{Real}(Bounds{Real}[Bounds{Real}(-1000.0, 1000.0)], Function[var"#222#223"(), var"#224#225"()], Function[], "Schaffer")


As this is a benchmark problem, we have encapsulated this code into a function: 
```julia
function schafferProblem()
  schaffer = ContinuousProblem{Real}("Schaffer")

  f = x -> x[1] * x[1]
  g = x -> (x[1] - 2.0) * (x[1] - 2.0)

  addObjective(schaffer, f)
  addObjective(schaffer, g)
  addVariable(schaffer, Bounds{Real}(-1000.0, 1000.0))

  return schaffer
end

```

To get the elements of a problem created in this way, the following functions are provided:
```julia

function numberOfVariables(problem::ContinuousProblem{T}) where {T}
  return length(problem.bounds)
end

function numberOfObjectives(problem::ContinuousProblem{T}) where {T}
  return length(problem.objectives)
end

function numberOfConstraints(problem::ContinuousProblem{T}) where {T}
  return length(problem.constraints)
end

function name(problem::ContinuousProblem{T}) where {T}
  return problem.name
end

function bounds(problem::ContinuousProblem{T}) where {T}
  return problem.bounds
end

```

In [11]:
println("Number of variables: ", numberOfVariables(schaffer))
println("Number of objectives: ", numberOfObjectives(schaffer))
println("Number of constraints: ", numberOfConstraints(schaffer))
println("Name: ", name(schaffer))
println("Bounds: ", bounds(schaffer))

Number of variables: 1
Number of objectives: 2
Number of constraints: 0
Name: Schaffer
Bounds: Bounds{Real}[Bounds{Real}(-1000.0, 1000.0)]


When a new solution is created, it must be evaluated to compute its objective values from the problem. The function for solution evaluations is common to all the problems created using the ``ContinuousProblem``struct: 
```julia
function evaluate(solution::ContinuousSolution{T}, problem::ContinuousProblem{T})::ContinuousSolution{T} where {T<:Number}
  for i in 1:length(problem.objectives)
    solution.objectives[i] = problem.objectives[i](solution.variables)
  end

  for i in 1:length(problem.constraints)
    solution.constraints[i] = problem.constraints[i](solution.variables)
  end

  return solution
end

```

## Method 2: Creating a specific struct for the problem

There are problems whose functions are not independent (e.g., the formulation of a function includes other function), so the approach of using the ``ContinuousProblem`` struct is not apropriate. An example is the ZDT6 problem, in which the second objective contains the value of evaluation the first one:

$$
f_1(\vec{x}) = 1 - exp(-4x_1)sin^6(6{\pi}x_1) 
$$
$$
f_2(\vec{x}) = g(\vec{x})[1 - (f_1(\vec{x})/g(\vec{x}))^2]
$$
$$
g(\vec{x}) = 1 + 9[(\sum_{i=2}^{n}x_i)/(n - 1)]^0.25
$$
$$
x \in [0.0, 1.0]
$$

This problem can be implemented as follows. First, we create a struct called ``ZDT6`` to contain the problem bounds:
    
```julia
struct ZDT6 <: AbstractContinuousProblem{Float64}
  bounds::Vector{Bounds{Float64}}
end
```

Next we define a function called ``zdt6Problem()`` which initializes the problem bounds and the set of functions to retrieve the problem data:

```julia
function zdt6Problem(numberOfVariables::Int=10)
  bounds = [Bounds{Float64}(0.0, 1.0) for _ in range(1, numberOfVariables)]

  return ZDT6(bounds)
end

function numberOfVariables(problem::ZDT6)
  return length(problem.bounds)
end

function numberOfObjectives(problem::ZDT6)
  return 2
end

function numberOfConstraints(problem::ZDT6)
  return 0
end

function name(problem::ZDT6)
  return "ZDT6"
end

```

The last step is to implement the ``evaluate`` method containing the code of the objective functions:
```julia
function evaluate(solution::ContinuousSolution{Float64}, problem::ZDT6)::ContinuousSolution{Float64}
  x = solution.variables
  @assert length(x) == numberOfVariables(problem) "The number of variables of the solution to be evaluated is not correct"

  function evalG(x::Vector{Float64})
    g = sum(x[i] for i in range(2,length(x)))
    g = g / (length(x) - 1.0)

    g = ^(g, 0.25)
    g = 9.0 * g
    g = 1.0 + g

    return g
  end

  function evalH(v::Float64, g::Float64)
    return 1.0 - ^(v/g, 2.0)
  end

  f1 = 1.0 - exp(-4.0*x[1]) * ^(sin(6*pi*x[1]),6.0)
  g = evalG(x)
  h = evalH(f1, g)
  f2 = g * h
 
  solution.objectives = [f1, f2]

  return solution
end
```